In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

%run ../modules/paper_figures.ipynb

In [ ]:
path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/23_t_Cys.xlsx'
df_wild = load_data(path)
path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/24_t_Cys_A.xlsx'
df_alkb = load_data(path)
df_wild.shape, df_alkb.shape

In [ ]:
df_wild = df_wild[df_wild.Mass>1500]
df_alkb = df_alkb[df_alkb.Mass>1500]
df_wild.shape, df_alkb.shape

In [ ]:
_, df_alkb_only = diff_dfs(df_wild, df_alkb)
df_alkb_only.shape

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
# params = [(df_intact_wild2, df_intact_alkb2_only, shift) for shift in np.arange(-64, 64, 1)]
params = [(df_wild, df_alkb_only, shift) for shift in np.arange(-64, 64, 1)]
# params = [(df_comm_wild2, df_comm_wild2, shift) for shift in np.arange(1, 64, 1)]
# params = [(df_wild2, df_alkb2, shift) for shift in np.arange(1, 100, 1)]
# params = [(df_sample, shift) for shift in np.arange(1, 64, 0.1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)    

In [ ]:
# df_sh = df_shift_hits.copy()
# df_sh.to_excel('Shift_Cys.xlsx')

In [ ]:
df_sh_top = df_shift_hits.sort_values('hit', ascending=False).reset_index(drop=True).iloc[:10].copy()

In [ ]:
def func(shift, df_s1, df_s2):
    dfm_l, dfm_r = peer_dfs(df_s1, df_s2, shift=shift)
    df = dfm_l if dfm_l.shape[0] < dfm_r.shape[0] else dfm_r
    bcr = base_calling_random(df)
    return len(bcr[1])

df_s1 = df_wild.copy()
df_s2 = df_alkb_only.copy()
# df_s1 = df_glu2.copy()
# df_s2 = df_glu2_alkb_only.copy()
df_sh_top['basecallings'] = df_sh_top.apply(lambda x: func(x['shift'], df_s1, df_s2), axis=1)

In [ ]:
fig = plt.figure(figsize=(2.5, 1.5))
df_plt = df_sh_top.copy().sort_values('shift')
df_plt1 = df_plt.copy()
df_plt1['shift'] = df_plt1['shift'].astype(int)
df_plt1 = df_plt1.set_index('shift')
df_plt1.plot(kind='bar',figsize=(2.5, 1.5))
plt.legend()

fig.tight_layout()
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
plt.savefig(svg_fpath, transparent=True, dpi=300)

In [ ]:
svg_fpath = '/Users/xyuan/Downloads/demo.svg'
draw_df(df_shift_hits[df_shift_hits.hit>0], svg_fpath, x='shift', y='hit')